In [17]:
# !pip install selenium webdriver-manager requests pykakao pywin32

In [18]:
# !pip install pycaw comtypes

^C


In [31]:
# edit date : 2024-03-05
# version : 1.9.0

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import *
from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import (
    StaleElementReferenceException,
    ElementClickInterceptedException,
    TimeoutException
)

import time
import datetime as dt
import random
from random import randint
import pandas as pd

import os
import win32api
import socket

from PyKakao import Message
import requests
import webbrowser
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
import time


def now():
    return dt.datetime.now()


## System Volumne Max________________________________________________________________________________    
def manage_volume(mode="set_max", saved_volume=None):
    """
    시스템 볼륨을 관리하는 함수.

    Args:
        mode (str): "set_max" -> 볼륨을 100%로 설정, "restore" -> 이전 볼륨으로 복원.
        saved_volume (float): 이전 볼륨 값을 저장하는 변수 (0.0 ~ 1.0 범위).
    
    Returns:
        float: 현재 볼륨 (mode="set_max"에서 저장된 볼륨 반환).
    """
    # 현재 오디오 장치를 가져옴
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = interface.QueryInterface(IAudioEndpointVolume)
    
    if mode == "set_max":
        # 현재 볼륨 저장
        current_volume = volume.GetMasterVolumeLevelScalar()
        print(f"Saved Volume: {current_volume * 100:.0f}%")
        
        # 볼륨을 100%로 설정
        volume.SetMasterVolumeLevelScalar(1.0, None)
        # print("Volume set to 100%.")
        return current_volume
    
    elif mode == "restore" and saved_volume is not None:
        # 저장된 볼륨으로 복원
        volume.SetMasterVolumeLevelScalar(saved_volume, None)
        print(f"Volume restored to {saved_volume * 100:.0f}%.")
    else:
        print("Invalid mode or no saved volume to restore.")


        
## kakao Message API Function________________________________________________________________________________
def kakaoMessage(type_contents=None, count=None):
    rest_api = 'c7331f1eedd34ad746c501eb91d59e1f'
    redirect_url = 'https://localhost:5000'
    token_url = 'https://kauth.kakao.com/oauth/token'
   

    # default Info
    init_token = ' GJCcB9q7J9WuOBWwU4DPjUMN2IOwW2zMAAAAAQorDNIAAAGTPw8rmtEMsmlHt4Ko'
    refresh_token = 'AmigbVWvdM-pMDqGIfn0rB1IMvi2KrWuAAAAAgorDNIAAAGTPw8rl9EMsmlHt4Ko'

    # token 만료 확인
    token_info_url = "https://kapi.kakao.com/v1/user/access_token_info"
    token_info_headers = {
            "Authorization": "Bearer " + init_token
        }
    toke_info_response = requests.get(token_info_url, headers=token_info_headers)
    
    # 아직 유효
    if toke_info_response.status_code==200:
        valid_access_token = init_token
    else:
        re_data = {
            "grant_type" : "refresh_token",
            "client_id" : rest_api,
            "refresh_token" : refresh_token
        }
        re_response = requests.post(token_url,data=re_data)
        re_token = re_response.json()
        valid_access_token = re_token['access_token']

    api = Message(service_key  = rest_api)
    api.set_access_token(valid_access_token)
    message_type = "text"
    
    # UUID 목록
    receiver_uuids = [
        "NwU9DDgKPgo4CSURIxMlEisbKBk1BDQCOg00AE8", 
    ]

    if type_contents:
        # 파라미터
        text = f"SRT 등록 완료 by 남기원 {count}"
        link = {
                    "web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000",
                    "mobile_web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000"
                }
        button_title = "바로 확인"
    else:
        text = f"매크로 체크용 : No.{count}"
        link = {
                    "web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000",
                    "mobile_web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000"
                }
        button_title = "바로 확인"
    
    api.send_message_to_friend(
        message_type=message_type, 
        receiver_uuids=receiver_uuids,
        text=text,
        link=link,
        button_title=button_title,
    )

    
## Chrome Driver 불러오기________________________________________________________________________________
def get_driverByPath(pathDriver):
    # pathDriver = r"C:\Users\hanta\.wdm\drivers\chromedriver\win64"
    subfolders = [f.name for f in os.scandir(pathDriver) if f.is_dir()]
    downloaded_latest_driver_version = max(subfolders)
    
    try:
        driver_path = rf"{pathDriver}\{downloaded_latest_driver_version}\chromedriver-win32\chromedriver.exe"
        driver = webdriver.Chrome(service=ChromeService(executable_path=driver_path))
        return driver
    except:
        driver_path = ChromeDriverManager().install()
        driver = webdriver.Chrome(service=ChromeService(executable_path=driver_path))
        return driver


## Login________________________________________________________________________________
def ExcutionbyDriver(input_driver, member_number, password, arrival, departure, standard_date, standard_time):
    ## login
    driver = input_driver
    driver.get('https://etk.srail.co.kr/cmc/01/selectLoginForm.do')
    driver.implicitly_wait(15)
    driver.find_element(By.ID, 'srchDvNm01').send_keys(member_number)
    driver.find_element(By.ID, 'hmpgPwdCphd01').send_keys(password)
    driver.find_element(By.XPATH, '/html/body/div/div[4]/div/div[2]/form/fieldset/div[1]/div[2]/div[2]/div/div[2]/input').click()
    driver.implicitly_wait(5)
    time.sleep(0.5)
    ## Start Search
    driver.get('https://etk.srail.kr/hpg/hra/01/selectScheduleList.do')
    driver.implicitly_wait(5)
    
    # 출발지 입력
    time.sleep(0.5)
    dep_stn = driver.find_element(By.ID, 'dptRsStnCdNm')
    dep_stn.clear()
    dep_stn.send_keys(arrival)

    # 도착지 입력
    time.sleep(0.5)
    arr_stn = driver.find_element(By.ID, 'arvRsStnCdNm')
    arr_stn.clear()
    arr_stn.send_keys(departure)

    # 날짜 드롭다운 리스트 보이게
    # elm_dptDt = driver.find_element(By.ID, "dptDt")
    # driver.execute_script("arguments[0].setAttribute('style','display: True;)", elm_dptDt)
    time.sleep(0.5)
    Select(driver.find_element(By.ID,"dptDt")).select_by_value(standard_date)

    # 출발 시간
    # eml_dptTm = driver.find_element(By.ID, "dptTm")
    # driver.execute_script("arguments[0].setAttribbute('style','display:True;')", eml_dptTm)
    time.sleep(0.5)
    Select(driver.find_element(By.ID, "dptTm")).select_by_visible_text(standard_time)

    # 조회하기 버튼
    driver.find_element(By.XPATH, "//input[@value='조회하기']").click()
    driver.implicitly_wait(1000)
    time.sleep(1)

## SRT reserving Function________________________________________________________________________________
def Repeat_reserved(input_driver, start_of_trains, number_of_trains):
    chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s'
    
    driver = input_driver
    reserved = False
    count = 0
    history  = []
    prev_fg = None
    prev_entey = None
    curr_fg = None
    
    while True:  
        dtm_step_01 = now()
        # print("------"*5)
        # print(f"A. 시작 : {dtm_step_01}")
        count +=1
        
        for i in range(start_of_trains, number_of_trains+1):
            try:
                standard_seat = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(7)").text
                reserved_seat = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(8)").text
            except StaleElementReferenceException:
                standard_seat = "매진"
                reserved_seat = "매진"
            
            # print(f"  01. Data Check({i}) : {reserved}, standard_seat= {standard_seat}, reserved_seat= {reserved_seat}", flush=True)
            
            ## 예약하기 진행
            if not reserved:
                # dtm_step_03 = now()
                # print(f"  02. 예약하기 진행 : {dtm_step_03} ({(dtm_step_03-dtm_step_01).total_seconds():.3f}s)", flush=True)
                if "예약하기" in standard_seat:
                    print(f"  03. 예약 가능 클릭 : {now()}", end='\n', flush=True)
                    try:
                        driver.find_element(By.XPATH, f"/html/body/div[1]/div[4]/div/div[3]/div[1]/form/fieldset/div[6]/table/tbody/tr[{i}]/td[7]/a/span").click()
                    except ElementClickInterceptedException as err:
                        print(err)
                        driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(7) > a").send_keys(Keys.ENTER)
                    finally:
                        driver.implicitly_wait(3)
                
                    # 예약 성공 시
                    if driver.find_elements(By.ID, 'isFalseGotoMain'):
                        reserved = True
                        print(f'  04-1. 예약 성공 : {now()}', end='\n', flush=True)
                        saved_volume = manage_volume(mode="set_max")
                        win32api.MessageBox(0, "Seat Reserved", "Done", 64)
                        manage_volume(mode="restore", saved_volume=saved_volume)
                        webbrowser.get(chrome_path).open("https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000")
                        break
                    else:
                        print(f"  04-2. 잔여석 없음. 다시 검색 : {now()}", end='\n', flush=True)
                        curr_fg = True
                        driver.back() #뒤로가기
                        driver.implicitly_wait(5)

                # 예약하기 가능할 경우
                if "신청하기" in reserved_seat:
                    driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(8) > a").click()
                    reserved = True
                    print(f"  04-3. 예약 대기 완료 : {now()}", end='\n', flush=True)
                    break

        if reserved:
            return reserved
            
        # 대기
        dtm_step_04 = now()
        # print(f"B. 대기 : {dtm_step_04} ({(dtm_step_04-dtm_step_01).total_seconds():.3f}s)", flush=True)
        randomndelay=random.randrange(2209,4432)/1000
        time.sleep(randomndelay)
        
        # 다시 조회
        submit = driver.find_element(By.XPATH, "//input[@value='조회하기']")
        driver.execute_script("arguments[0].click();", submit)
        dtm_step_05 = now()
        # print(f"▶ 새로고침 {count} : {dtm_step_05} ({(dtm_step_05-dtm_step_04).total_seconds():.3f}s)", flush=True)
        driver.implicitly_wait(1)


        # 결과를 history에 쌓기
        entry = {
            'cycle': count,
            'start_time': dtm_step_01.strftime("%H:%M:%S"),
            'refresh_time': dtm_step_05.strftime("%H:%M:%S"),
            'delay_s': round(randomndelay, 3),
            'fg': curr_fg
        }
        history.append(entry)

       # fg가 바뀌었을 때만 테이블(마지막 5개 행) 출력
        if prev_fg is not None and curr_fg !=prev_fg:
            df = pd.DataFrame([prev_entry, entry])
            print("\n=== fg 변경 감지! 최근 사이클 결과 ===")
            print(df.to_string(index=False))
        else:
            # 평소: 최신 한 줄만 출력
            df = pd.DataFrame([entry])
            print(df.to_string(index=False, header=False), end='\r')
            
        prev_fg = curr_fg
        prev_entry = entry

        
        # 대기자가 있을 시 대기
        dtm_step_06 = now()
        # print(f"C. 대기자 있을 시 대기 : {dtm_step_06} ({(dtm_step_06-dtm_step_05).total_seconds():.3f}s)", flush=True)
        css = "#NetFunnel_Loading_Popup_Count"
        elems = driver.find_elements(By.CSS_SELECTOR, css)
        disappear_timeout = 60 # 15분
        if elems:
            popup = elems[0]
            print(f"Waiting Popup's Shown. Waiting Untill disapear : {now()}", flush=True)
            try:
                # WebDriverWait(driver, disappear_timeout).until(EC.staleness_of(popup))
                # 1초 동안만 팝업 존재 확인
                WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
                # 팝업이 보이면 최대 600초 기다려서 사라질 때까지 대기
                WebDriverWait(driver, 600).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, css)))
            except TimeoutException:
                print(f"Time out for awaiting {disappear_timeout}s, Refresh Process : : {now()}", flush=True)
                driver.refresh()

        
print(f'done {now()}')

done 2025-06-16 11:27:26.462364


## manage_volume test
saved_volume = manage_volume(mode="set_max")
# 몇 초 후 원래 볼륨으로 복원
import win32api

time.sleep(1)
win32api.MessageBox(0, "Seat Reserved", "Done", 64)
manage_volume(mode="restore", saved_volume=saved_volume)

In [30]:
# Reservation info.
#=============================================================================
member_number = "2092236381" # 회원번호
password = "thfldjqtdl2@" # 비밀번호
arrival = "대전" # 출발지
departure = "수서" # 도착지
standard_date = "20250620" # 기준날짜 ex) 20221101
standard_time = "14" # 기준 시간 ex) 00 - 22 // 2의 배수로 입력
start_of_trains = 1 # 검색 시 보여지는 기차 중 시작할 기차의 서수
number_of_trains = 2 # 상단에서부터 조회할 기차수  maximum = 10
#=============================================================================


## Desktop, Path Setting
current_desktop = socket.gethostname()
print(f"Desktop Name : {current_desktop}")

# 폴더 경로 지정
if 'OCL' in current_desktop :
    folder_path = r"C:\Users\hanta\Desktop\study\python/local-home-4090"
    pathDriver = r"C:\Users\hanta\.wdm\drivers\chromedriver\win64"
else :
    folder_path = r"C:\Users\Administrator\Desktop\code/local-home-4090"
    pathDriver = r"C:\Users\Administrator\.wdm\drivers\chromedriver\win64"
print(f"Folder_path : {folder_path} \nDriver_path : {pathDriver}")


##______________________________________________________________
driver = get_driverByPath(pathDriver)
ExcutionbyDriver(driver, member_number, password, arrival, departure, standard_date, standard_time)
Repeat_reserved(driver, start_of_trains, number_of_trains)

Desktop Name : OCL2208007
Folder_path : C:\Users\hanta\Desktop\study\python/local-home-4090 
Driver_path : C:\Users\hanta\.wdm\drivers\chromedriver\win64
13	11:27:16	11:27:19	3.581	None

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.104)
Stacktrace:
	GetHandleVerifier [0x0xd93763+63299]
	GetHandleVerifier [0x0xd937a4+63364]
	(No symbol) [0x0xbc1113]
	(No symbol) [0x0xbb08a0]
	(No symbol) [0x0xbce8c2]
	(No symbol) [0x0xc3519c]
	(No symbol) [0x0xc4f359]
	(No symbol) [0x0xc2e376]
	(No symbol) [0x0xbfd6e0]
	(No symbol) [0x0xbfe544]
	GetHandleVerifier [0x0xfee033+2531347]
	GetHandleVerifier [0x0xfe9332+2511634]
	GetHandleVerifier [0x0xdb9eda+220858]
	GetHandleVerifier [0x0xdaa528+156936]
	GetHandleVerifier [0x0xdb0c5d+183357]
	GetHandleVerifier [0x0xd9b6c8+95912]
	GetHandleVerifier [0x0xd9b870+96336]
	GetHandleVerifier [0x0xd8664a+9770]
	(No symbol) [0x0x76297ba9]
	RtlInitializeExceptionChain [0x0x770ec36b+107]
	RtlClearBits [0x0x770ec2ef+191]


In [37]:
t1 = now()
time.sleep(1.1)
t2 = now()
print(t1,t2)
print((t2-t1).total_seconds())

2025-06-13 13:33:43.675069 2025-06-13 13:33:44.775784
1.100715


In [84]:
for i in range(2,2+1):
    print(i)

2


In [32]:
dt.datetime.now()

datetime.datetime(2025, 6, 13, 13, 32, 30, 897470)

In [ ]:
## Github Setting
# git remote 명령어 실행
# import subprocess
# remote_url = "https://github.com/your_username/your_repository.git"
# subprocess.run(["git", "-C", folder_path, "remote", "add", "origin", remote_url])

# git pull 명령어 실행
# subprocess.run(["git", "-C", folder_path, "pull"])